In [1]:
import os
from copy import deepcopy
from tqdm import tqdm
from datetime import date, datetime
import pandas as pd
import numpy as np

In [2]:
df_all = []
file_lst_used = []
file_lst = os.listdir('NMXLNT')
for file_name in file_lst:
    if "VINH NIEM" not in file_name and "HO TAY" not in file_name:
        df = pd.read_csv(f"NMXLNT/{file_name}")
        df = df.drop_duplicates().reset_index().drop("index", axis = 1) # after some inspection, realize there's some duplicates
        df_all.append(df)
        file_lst_used.append(file_name)
        print(file_name)
        print(df.columns)

BAY MAU.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
TU SON.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
CAU NGA.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
BTLVT.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
YENSO.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
 

In [3]:
time_fraction_used = [i/2 for i in range(2)]
lag_used = [i + t for t in time_fraction_used for i in range(4, 12)] + [12]
actual_lag_used = [round(i + t, 2) for t in time_fraction_used for i in range(4, 12)] + [12]
features_used = [f"cod_prev_{i}" for i in actual_lag_used] + \
                [f"temp_prev_{i}" for i in actual_lag_used] + \
                [f"ph_prev_{i}" for i in actual_lag_used] + \
                [f"tss_prev_{i}" for i in actual_lag_used] + \
                [f"flow_out_prev_{i}" for i in actual_lag_used] + \
                [f"nh4_prev_{i}" for i in actual_lag_used] + \
                ["sin_hour", "sin_day", "sin_month", "cod"] + \
                ["datetime", "Location"]

# try to plot cod and find error data in this
def find_prev_mean(row, df_copy, num_minutes):
    # find the mean of last num_minutes minutes
    begin_time, end_time = row["datetime"] - pd.Timedelta(minutes = num_minutes), row["datetime"]
    past_mean = df_copy[(df_copy["datetime"] >= begin_time) & (df_copy["datetime"] < end_time)]["cod"].mean()
    if pd.isna(past_mean) or past_mean == 0:
        return True
    return abs(row["cod"] / past_mean - 1) > 0.2

def preprocessing(df, file_name):
    df["Location"] = file_name
    df["datetime"] = pd.to_datetime(df["datetime"])
    df["minute"] = df["datetime"].dt.minute
    df["sin_minute"] = np.sin(df["minute"] / 60 * 2 * np.pi)
    df["hour"] = (df["datetime"].dt.hour)
    df["sin_hour"] = np.sin(df["hour"] / 24 * 2 * np.pi)
    #df["dayofweek"] = (df["datetime"].dt.dayofweek)
    df["day"] = (df["datetime"].dt.day)
    df["sin_day"] = np.sin(df["day"] / 31 * 2 * np.pi)
    df["month"] = (df["datetime"].dt.month)
    df["sin_month"] = np.sin(df["month"] / 12 * 2 * np.pi)
    df = df[~df["cod"].isna()].reset_index().drop("index", axis = 1)
    df = df[df["cod"] > 0].reset_index().drop("index", axis = 1) # assumption of cod > 0
    if "flow_in" not in df.columns:
        df["flow_in"] = df["flow_in1"] + df["flow_in2"]
    df["flow_out"] = df["flow_out1"].fillna(0) + df["flow_out2"].fillna(0) + df["flow_out3"].fillna(0)
    #print(df["cod"].isna().sum())
    df = df.reset_index().drop("index", axis = 1)
    temp_df = deepcopy(df)
    last_row_with_time_within = {
        j: [] for j in lag_used # store between 4 - 8 hours since we are trying to predict the next 4-8 hours
    }
    curr_row = {
        j: 0 for j in lag_used
    }
    for i in tqdm(range(df.shape[0])):
        for j in lag_used:
            # we first try to find until we find first instance that is STRICTLY LESS THAN k hours before current time
            # then the previous instance is the instance that might before around 4 hours before current time
            while curr_row[j] < i and (df.loc[i, "datetime"] - df.loc[curr_row[j], "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j:
                curr_row[j] += 1
            # when we stop is when we do not see a fit row, try to check if that fit row actually fit 
            # a fit would be from k - < k+1 hours before (curr_row - 1 would be our answer)
            if curr_row[j] <= i and curr_row[j] >= 1 and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) < 60 * (j+1):
                last_row_with_time_within[j].append(curr_row[j] - 1)
            else:
                # we stop because we have no answer
                last_row_with_time_within[j].append(None)
    for j in lag_used:
        df[f"last_row_with_time_within_{round(j, 2)}hour"] = last_row_with_time_within[j]
    # need to separete these two steps since after we inner join, we will lose some rows
    for j in lag_used:
        df2 = deepcopy(temp_df.reset_index())
        df = df.merge(df2, how = "inner", left_on = f"last_row_with_time_within_{round(j, 2)}hour", right_on = "index", suffixes=("", f"_prev_{round(j, 2)}"))
        #print(np.mean(((df["datetime"] - df[f"datetime_prev_{j}"]) / pd.Timedelta(minutes=1)) >= 60*j))
    # print(df.shape[0])
    # df["zscore"] = (df["cod"] - df["cod"].rolling(50).mean()) / df["cod"].rolling(50).std()
    # df = df[df["zscore"].abs() <= 6.5].reset_index().drop("index", axis = 1)
    # print(df.shape[0])
    temp = deepcopy(df)
    df["abnormal"] = df.apply(lambda x: find_prev_mean(x, temp, 30), axis = 1)
    df = df[~df["abnormal"]].reset_index().drop("index", axis = 1)
    df = df[features_used]
    return df

In [4]:
for i in range(len(df_all)):
    df_all[i] = preprocessing(df_all[i], file_lst_used[i])
df_all = pd.concat(df_all, ignore_index = True)
df_all.to_csv("NMXLNT_df.csv", index = False)

100%|██████████| 73175/73175 [00:55<00:00, 1308.80it/s]
